In [1]:
import pandas as pd
import numpy as np
import nltk
import math
from joblib import Parallel, delayed

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('rslp')

from nltk.tokenize import word_tokenize
from string import punctuation
import nltk
from unicodedata import normalize
from nltk.stem import RSLPStemmer
from nltk.util import ngrams
from nltk.tokenize import RegexpTokenizer
from sklearn.model_selection import train_test_split

from joblib import Parallel, delayed
import time


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Flavio\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Flavio\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\Flavio\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!


In [2]:
# df_assunto= pd.read_csv("data/dados-conle-anonimizado-assunto-notnull.csv", encoding='utf-8')
df_assunto= pd.read_table("dados-conle-anonimizado-assunto-notnull - dados-conle-anonimizado-assunto-notnull.tsv",sep=";")


In [3]:
df_assunto.columns

Index(['NÚMERO-PROPOSIÇÃOSILEG', 'TxtAssunto\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t'], dtype='object')

In [4]:
base = pd.read_csv("base_20230428_douglas.csv", sep=',')
# base = pd.read_csv("data/proposicao-tema-completo-sem-duplicado-2019-all-fields.csv", sep=",")

In [5]:
base

,codProposicao,txtSiglaTipo,numAno,numNumero,txtNome,txtEmenta,txtExplicacaoEmenta,txtIndexacao,txtInteiroTeor
0,16357,PL,1999,1165,PL 1165/1999,"Altera dispositivo da Lei nº 8.987, de 13 de f...",Estabelece que as concessionárias disponibiliz...,"Alteração, Lei das Concessões de Serviços Públ...",Ofício nº 1416 (SF) ...
1,19098,PL,1992,3097,PL 3097/1992,Dispõe sobre a eleição de diretores de fundos ...,NaN,"NORMAS, ELEIÇÃO DIRETA, EMPREGADO, APOSENTADO,...",COMISSÃO DE CONSTITUIÇÃO E JUSTIÇA E DE REDAÇÃ...
2,20464,PL,2000,3927,PL 3927/2000,Altera a composição dos Tribunais Regionais do...,"Altera a composição do TRT da 5ª região, 6ª re...","Alteração, Lei Federal, composição, Tribunal R...","COMISSÃO DE TRABALHO, DE ADMINISTRAÇÃO E SERVI..."
3,20683,PL,1998,4117,PL 4117/1998,Dispõe sobre o acesso a ambientes de uso colet...,NaN,"Autorização, pessoa portadora de deficiência, ...",Câmara dos Deputados\n Departamento ...
4,20857,PL,1998,4395,PL 4395/1998,Estabelece as Diretrizes Nacionais de Defesa C...,NaN,"Fixação, diretrizes, defesa civil, (Sindec), d...",- 1 - \nCOMISSÃO DE CONSTITUIÇÃO E JUSTIÇA E D...
...,...,...,...,...,...,...,...,...,...
57104,2358873,PL,2023,2233,PL 2233/2023,Altera o parágrafo único do art. 71 do Código ...,NaN,NaN,1 \n \n \n \n \nCÂMARA DOS DEPUTADOS \nPROJET...
57105,2358874,PL,2019,3616,PL 3616/2019,"Altera a Lei nº 9.503, de 23 de setembro de 19...",NaN,NaN,"Altera a Lei nº 9.503, de 23 de setembro\nde 1..."
57106,2358875,PL,2019,1822,PL 1822/2019,"Altera a Lei nº 11.340, de 7 de agosto de 2006...",NaN,NaN,"Altera a Lei nº 11.340, de 7 de agosto de\n200..."
57107,2358877,PL,2019,3815,PL 3815/2019,"Altera a Lei nº 7.565, de 19 de dezembro de 19...",NaN,NaN,"Altera a Lei nº 7.565, de 19 de dezembro\nde ..."


In [6]:
df_assunto.iloc[0,1].strip()

'Solicito a consultoria elaboração de Projeto de lei para garantir a estabilidade da mulher gravida quando esteja em cargo publico comissionado.   Como não é possível impedir a exoneração que já preveja uma indenização no valor da media salarial do ultimo ano durante o período em que está gravida e período da licença maternidade.'

In [7]:
base.columns

Index(['codProposicao', 'txtSiglaTipo', 'numAno', 'numNumero', 'txtNome',
       'txtEmenta', 'txtExplicacaoEmenta', 'txtIndexacao', 'txtInteiroTeor'],
      dtype='object')

In [8]:
data = base.txtInteiroTeor.dropna()        # Trocar com nome da coluna desejada
data = data.to_numpy()

In [9]:
arr_assunto = df_assunto.to_numpy()
y,X = arr_assunto[:,0],arr_assunto[:,1]
y = [i.strip() for i in y]

### BM25

In [10]:
import math
import numpy as np
import sys
from multiprocessing import Pool, cpu_count
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

DEFAULT_CUT = 0.4
DEFAULT_DELTA = 0.1

class BM25:
    def __init__(self, corpus, tokenizer=None):
        self.corpus_size = len(corpus)
        self.avgdl = 0
        self.idf = {}
        self.doc_len = []
        self.tokenizer = tokenizer
        self.term_freqs = {} # dicionário para guardar os "term frequencies" (TF), da forma {'termo': [lista de frequências onde freq > 0], ...}
        self.term_docs = {} # dicionário para guardar os documentos onde ocorre cada termo {'termo': [lista de docs onde ocorre 'termo'], ...}

        if tokenizer:
            corpus = self._tokenize_corpus(corpus)

        nd = self._initialize(corpus)
        self._calc_idf(nd)

    def _initialize(self, corpus):
        num_doc = 0
        doc_n = 0  # nº do documento
        for document in corpus:
            self.doc_len.append(len(document))
            num_doc += len(document)

            # dicionário para controlar quebra de documento para cada n-grama
            doc_changed = {}
            for word in document:
                doc_changed[sys.intern(word)] = True # significa que mudou o documento; indica para todas as palavras do documento

            for word in document:
                # Incrementa o dicionário de TF
                if word not in self.term_freqs:
                    self.term_freqs[sys.intern(word)] = [1]
                    self.term_docs[sys.intern(word)] = [doc_n]
                    doc_changed[sys.intern(word)] = False
                else:
                    if not doc_changed[sys.intern(word)]:
                        self.term_freqs[sys.intern(word)][-1] += 1  # incrementa frequência do termo no documento = TF(t, d)
                    else:
                        self.term_freqs[sys.intern(word)].append(1)
                        self.term_docs[sys.intern(word)].append(doc_n)
                        doc_changed[sys.intern(word)] = False
            doc_n += 1

        # monta o dicionário DF (Document Frequency)
        df = {}
        for term_freq, lst_freqs in zip(self.term_freqs.keys(), self.term_freqs.values()):
            df[sys.intern(term_freq)] = len(lst_freqs)

        self.avgdl = num_doc / self.corpus_size
        return df

    def _tokenize_corpus(self, corpus):
        pool = Pool(cpu_count())
        tokenized_corpus = pool.map(self.tokenizer, corpus)
        return tokenized_corpus

    def _calc_idf(self, nd):
        raise NotImplementedError()

    def get_scores(self, query):
        raise NotImplementedError()

    def get_batch_scores(self, query, doc_ids):
        raise NotImplementedError()

    def get_top_n(self, query, documents, n=5, 
                    improve_similarity=False, raw_query=None, past_queries=[], 
                    retrieved_docs=[], names=[], cut=DEFAULT_CUT, delta=DEFAULT_DELTA):

        assert self.corpus_size == len(documents), "The documents given don't match the index corpus"

        scores = self.get_scores(query)
        try:
            scores_normalized = (scores - np.min(scores)) / (np.max(scores) - np.min(scores))
        except:
            scores_normalized = [0 for i in range(scores)]

        scores_final = None
        if (improve_similarity):
            try:
                lambdas = self._lambda_calc(all_queries=past_queries, retrieved_docs=retrieved_docs,
                                            query=raw_query, cut=cut, delta=delta)
                scores_final = self._lambda_update(scores=scores_normalized, lambdas=lambdas, names=names)
            except:
                print("Error calculating lambdas. If there are no past feedbacks yet ignore this message.", flush=True)
                scores_final = scores

        top_n = np.argsort(scores_final)[::-1][:n]
        return [documents[i] for i in top_n], np.sort(scores)[::-1][:n], np.sort(scores_normalized)[::-1][:n], np.sort(
            scores_final)[::-1][:n]

### BM25L

In [11]:
class BM25L(BM25):
    def __init__(self, corpus, tokenizer=None, k1=1.5, b=0.75, epsilon=0.25):
        self.k1 = k1
        self.b = b
        self.epsilon = epsilon
        super().__init__(corpus, tokenizer)

    # Calculo do IDF (Inverse Document Frequency)
    def _calc_idf(self, nd):
        # collect idf sum to calculate an average idf for epsilon value
        idf_sum = 0
        # collect words with negative idf to set them a special epsilon value.
        # idf can be negative if word is contained in more than half of documents
        negative_idfs = []
        for word, freq in nd.items():
            idf = math.log(self.corpus_size + 1) - math.log(freq + 0.5)
            self.idf[sys.intern(word)] = idf
            idf_sum += idf
            if idf < 0:
                negative_idfs.append(word)
        self.average_idf = idf_sum / len(self.idf)

        eps = self.epsilon * self.average_idf
        for word in negative_idfs:
            self.idf[sys.intern(word)] = eps

    # Calculo do ctd
    def get_ctd(self, q_freq, b, doc_len, avg_len):
        ctd = q_freq/(1 - b + b*(doc_len)/(avg_len))
        return ctd

    # Avaliar a pontuacao de todos os documentos na base
    def get_scores(self, query):
        score = np.zeros(self.corpus_size)
        doc_len = np.array(self.doc_len)

        # Funcionamento de term_freqs e term_docs
        # Ex: term_freqs['termo'] = [10, 5, 4, 15] => frequências do termo (TF > 0)
        #     term_docs['termo'] = [5, 20, 40, 55] => termo ocorre nos docs 5, 20, 40, 55 
        for q in query:
            if q not in self.term_freqs:
                continue
            q_tf = [0]*self.corpus_size 
            for docn, tf in zip(self.term_docs[q], self.term_freqs[q]):
                q_tf[docn] = tf
            ctd = q_tf / (1 - self.b + self.b * (doc_len) / (self.avgdl))
            score += (self.idf.get(q, 0)) * ((ctd + 0.5) * (self.k1 + 1) / ((ctd + 0.5) + self.k1))

        return score


    def _lambda_update(self, scores, lambdas, names):
        """
        Updates bm25 scores using the lambdas values
        """
        result = np.copy(scores)
        for i, name in enumerate(names):
            name = name.strip()
            if (name in lambdas.keys()):
                result[i] += lambdas[name]
        return result

    def _lambda_calc(self, all_queries, retrieved_docs, query, cut, delta):
        """
        Searches for similar queries; returns dictionary
        """
        vectorizer = TfidfVectorizer()
        vectorizer.fit(all_queries+[query])
        vsm_2 = vectorizer.transform(all_queries)
        vsm_1 = vectorizer.transform([query])
        similarities = cosine_similarity(vsm_1, vsm_2).tolist()[0]

        doc_sim = [(retrieved_docs[j], similarities[j]) for j in range(len(similarities)) if similarities[j] > cut]

        dic = {}
        for tuple in doc_sim:
            sim = tuple[1]
            for doc in tuple[0]:
                (name, score, score_norm, relevance) = doc
                rel = 1
                if (relevance == 'i'):
                    rel = -1

                if (name in dic):
                    dic[name] += score_norm * sim * rel
                else:
                    dic[name] = score_norm * sim * rel # calculando a soma do produto sim*score*rel

        for key in dic:
            dic[key] = np.log(dic[key] + 1) * delta
        return dic

    def get_top_n(self, query, documents, n=5,
                  improve_similarity=False, raw_query=None, past_queries=[],
                  retrieved_docs=[], names=[], cut=0.6, delta=0.7):

        assert self.corpus_size == len(documents), "The documents given don't match the index corpus"

        scores = self.get_scores(query)

        if np.isclose(np.max(scores), np.min(scores), atol=1e-5):
            score_ref = 1.0 if np.max(scores) > 1e-6 else 0.0
            scores_normalized = np.array([score_ref for i in range(len(scores))])
        else:
            scores_normalized = (scores - np.min(scores)) / (np.max(scores) - np.min(scores))

        scores_final = np.copy(scores_normalized)
        if (improve_similarity and len(past_queries) > 0):
            lambdas = self._lambda_calc(all_queries=past_queries, retrieved_docs=retrieved_docs,
                                        query=raw_query, cut=cut, delta=delta)
            scores_final = self._lambda_update(scores=scores_normalized, lambdas=lambdas, names=names)

        top_n = np.argpartition(scores_final, -n)[::-1][:n]
        top_n = top_n[np.argsort(scores_final[top_n])[::-1]]

        return [documents[i] for i in top_n]



### Savoy

In [12]:
class Savoy:

    def __removeAllPTAccent(self, old_word):
        word = list(old_word)
        len_word = len(word)-1
        for i in range(len_word, -1, -1):
            if word[i] == 'ä':
                word[i] = 'a'
            if word[i] == 'â':
                word[i] = 'a'
            if word[i] == 'à':
                word[i] = 'a'
            if word[i] == 'á':
                word[i] = 'a'
            if word[i] == 'ã':
                word[i] = 'a'
            if word[i] == 'ê':
                word[i] = 'e'
            if word[i] == 'é':
                word[i] = 'e'
            if word[i] == 'è':
                word[i] = 'e'
            if word[i] == 'ë':
                word[i] = 'e'
            if word[i] == 'ï':
                word[i] = 'i'
            if word[i] == 'î':
                word[i] = 'i'
            if word[i] == 'ì':
                word[i] = 'i'
            if word[i] == 'í':
                word[i] = 'i'
            if word[i] == 'ü':
                word[i] = 'u'
            if word[i] == 'ú':
                word[i] = 'u'
            if word[i] == 'ù':
                word[i] = 'u'
            if word[i] == 'û':
                word[i] = 'u'
            if word[i] == 'ô':
                word[i] = 'o'
            if word[i] == 'ö':
                word[i] = 'o'
            if word[i] == 'ó':
                word[i] = 'o'
            if word[i] == 'ò':
                word[i] = 'o'
            if word[i] == 'õ':
                word[i] = 'o'
            if word[i] == 'ç':
                word[i] = 'c'

        new_word = "".join(word)
        return new_word

    def __finalVowelPortuguese(self, word):
        len_word = len(word)
        if len_word > 3:
            if word[-1] == 'e' or word[-1] == 'a' or word[-1] == 'o':
                word = word[:-1]

        return word

    def __remove_PTsuffix(self, word):
        len_word = len(word)

        if len_word > 3:
            if word[-1] == 's' and word[-2] == 'e' and (word[-3] == 'r' or word[-3] == 's' or word[-3] == 'z' or word[-3] == 'l'):
                word = word[:-2]
                return word
        if len_word > 2:
            if word[-1] == 's' and word[-2] == 'n':
                new_word = list(word)
                new_word[-2] = 'm'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing

        if len_word > 3:
            if (word[-1] == 's' and word[-2] == 'i') and (word[-3] == 'e' or word[-3] == 'é'):
                new_word = list(word)
                new_word[-3] = 'e'
                new_word[-2] = 'l'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing

        if len_word > 3:
            if word[-1] == 's' and word[-2] == 'i' and word[-3] == 'a':
                new_word = list(word)
                new_word[-2] = 'l'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing

        if len_word > 3:
            if word[-1] == 's' and word[-2] == 'i' and word[-3] == 'ó':
                new_word = list(word)
                new_word[-3] = 'o'
                new_word[-2] = 'l'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing

        if len_word > 3:
            if word[-1] == 's' and word[-2] == 'i':
                new_word = list(word)
                new_word[-1] = 'l'
                sing = "".join(new_word)
                return sing

        if len_word > 2:
            if word[-1] == 's' and word[-2] == 'e' and word[-3] == 'õ':
                new_word = list(word)
                new_word[-3] = 'ã'
                new_word[-2] = 'o'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing
            if word[-1] == 's' and word[-2] == 'e' and word[-3] == 'ã':
                new_word = list(word)
                new_word[-2] = 'o'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing

        if len_word > 5:
            if word[-1] == 'e' and word[-2] == 't' and word[-3] == 'n' and word[-4] == 'e' and word[-5] == 'm':
                word = word[:-5]
                return word

        if len_word > 2:
            if word[-1] == 's':
                word = word[:-1]

        return word

    def __normFemininPortuguese(self, word):

        len_word = len(word)

        if len_word < 3 or word[-1] != 'a':
            return word

        if len_word > 6:

            if word[-2] == 'h' and word[-3] == 'n' and word[-4] == 'i':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 'c' and word[-3] == 'a' and word[-4] == 'i':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 'r' and word[-3] == 'i' and word[-4] == 'e':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

        if len_word > 5:
            if word[-2] == 'n' and word[-3] == 'o':
                new_word = list(word)
                new_word[-3] = 'ã'
                new_word[-2] = 'o'
                masc = "".join(new_word)
                masc = masc[:-1]
                return masc

            if word[-2] == 'r' and word[-3] == 'o':
                word = word[:-1]
                return word

            if word[-2] == 's' and word[-3] == 'o':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 's' and word[-3] == 'e':
                new_word = list(word)
                new_word[-3] = 'ê'
                masc = "".join(new_word)
                masc = masc[:-1]
                return masc

            if word[-2] == 'c' and word[-3] == 'i':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 'd' and word[-3] == 'i':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 'd' and word[-3] == 'a':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 'v' and word[-3] == 'i':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 'm' and word[-3] == 'a':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 'n':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

        return word

    def stem(self, word):
        len_word = len(word)
        if len_word > 2:
            word = self.__remove_PTsuffix(word)
            word = self.__normFemininPortuguese(word)
            word = self.__finalVowelPortuguese(word)
            word = self.__removeAllPTAccent(word)

        return word


In [13]:
class RSLP_S:
    def __plural_reduction(self, word):
        excep = ["lápis","cais","mais","crúcis","biquínis","pois","depois","dois","leis" ]
        excep_s = ["aliás","pires","lápis","cais","mais","mas","menos", "férias","fezes","pêsames","crúcis","gás", "atrás","moisés","através","convés","ês", "país","após","ambas","ambos","messias"]

        len_word = len(word)
        new_word = list(word)

        if len_word >= 3:
            if new_word[-1] == 's' and new_word[-2] == 'n':
                new_word[-2] = 'm'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing

            if new_word[-1] == 's' and new_word[-2] == 'e' and new_word[-3] == 'õ':
                new_word[-3] = 'ã'
                new_word[-2] = 'o'
                sing = "".join(new_word)
                sing = sing[:-1]
                return  sing

            if new_word[-1] == 's' and new_word[-2] == 'e' and new_word[-3] == 'ã':
                if word == 'mães':
                    word = word[:-1]
                    return word
                else:
                    new_word[-2] = 'o'
                    sing = "".join(new_word)
                    sing = sing[:-1]
                    return sing

            if new_word[-1] == 's' and new_word[-2] == 'i' and new_word[-3] == 'a':
                if word != 'cais' and word != 'mais':
                    new_word[-2] = 'l'
                    sing = "".join(new_word)
                    sing = sing[:-1]
                    return sing

            if new_word[-1] == 's' and new_word[-2] == 'i' and new_word[-3] == 'é':
                new_word[-3] = 'e'
                new_word[-2] = 'l'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing

            if new_word[-1] == 's' and new_word[-2] == 'i' and new_word[-3] == 'e':
                new_word[-3] = 'e'
                new_word[-2] = 'l'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing

            if new_word[-1] == 's' and new_word[-2] == 'i' and new_word[-3] == 'ó':
                new_word[-3] = 'o'
                new_word[-2] = 'l'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing

            if new_word[-1] == 's' and new_word[-2] == 'i':
                if word not in excep:
                    new_word[-1] = 'l'
                    sing = "".join(new_word)
                    return sing

            if new_word[-1] == 's' and new_word[-2] == 'e' and new_word[-3] == 'l':
                word = word[:-2]
                return word

            if new_word[-1] == 's' and new_word[-2] == 'e' and new_word[-3] == 'r':
                word = word[:-2]
                return word

            if new_word[-1] == 's':
                if word not in excep_s:
                    word = word[:-1]

        return word

    def stem(self, word):
        word = self.__plural_reduction(word)

        return word

### Avaliação

In [14]:
def get_name(base, doc):
    return str(base[base.txtInteiroTeor==doc].txtNome.to_numpy()[0]).strip()

In [12]:
def avaliacaoRR(top_k=20):
    primeira_posicao=0
    
    for l,x in zip(y,X):
        tokenized_query3 = preprocess(x)                   
     
        top_n_stem_l = bm25.get_top_n(tokenized_query3,data, n=top_k)
        labelsnsL = [get_name(base,d) for d in top_n_stem_l]
        if str(labelsnsL[0]).strip() == l.strip():
            primeira_posicao+=1
    rr = primeira_posicao / 295
    print("RR: %f" % (rr))

In [13]:
def avaliacaoMAP(top_k=20):
    
    l_v = list()
    labels_nsL = list()
    for l,x in zip(y,X):
        tokenized_query3 = preprocess(x)                   
        top_n_stem_l = bm25.get_top_n(tokenized_query3,data, n=top_k)
        labelsnsL = [get_name(base,d) for d in top_n_stem_l] 
        
        l_v.append(l)
        labels_nsL.append(labelsnsL)
    
    denominador = 0
    
    encontrou=False
    soma=0
    for Y,x in zip(l_v,labels_nsL):
        calc = 0
        quant=0
        for k in x:
            if str(k).strip() == Y.strip():
                quant=1
                encontrou=True
            denominador+=1
            calc=quant/denominador
            if encontrou:
                calc = calc * 1
            else:
                calc = calc * 0
            soma+=calc
            encontrou = False
    MAP = soma /295
    print("MAP: %f" % (MAP))            

In [14]:
def avaliacaoPrecision(top_k=20):
    l_v = list()
    labels_nsL = list()
    for l,x in zip(y,X):
        tokenized_query3 = preprocess(x)                   
        top_n_stem_l = bm25.get_top_n(tokenized_query3,data, n=top_k)
        labelsnsL = [get_name(base,d) for d in top_n_stem_l] 
        
        l_v.append(l)
        labels_nsL.append(labelsnsL)
    
    soma=0
    for Y,x in zip(l_v,labels_nsL):
        calc = 0
        for k in x:
            if str(k).strip() == Y.strip():
                calc=1
                break
        calc=calc/top_k
        soma+=calc
    precision = soma /295
    print("Precision: %f" % (precision))

In [15]:
def avaliacaoRecall(top_k=20):

    labels_nsL = list()


    l_v = list()
    antes = time.time()
    for l,x in zip(y,X):

        tokenized_query3 = preprocess(x)                   
     
        top_n_stem_l = bm25.get_top_n(tokenized_query3,data, n=top_k)
        
   
        labelsnsL = [get_name(base,d) for d in top_n_stem_l]              #L
 
        labels_nsL.append(labelsnsL)    #L
        
    
        l_v.append(l)
        
    
    res_nsL = list()                #Stopwords + Acentuação + Stemming + Unigram + Bigram (L)
    depois = time.time()
    print("Duração: %f" %(depois - antes))

    for k in range(1, top_k+1):


        resultsnsL = list()         #Stopwords + Acentuação + Stemming + Unigram + Bigram (L)


        for i in range(len(l_v)):


            if (l_v[i] in labels_nsL[i][:k]):
                resultsnsL.append(1)
            else:
                resultsnsL.append(0)

       
        res_nsL.append(sum(resultsnsL)/len(resultsnsL))
    print(f"Encontrados: {sum(resultsnsL)}")
    print(f"Total: {len(arr_assunto)}")
#     print(f"Acurácia 10 docs: {res_nsL[9]}")
    print(f"Acurácia 20 docs: {res_nsL[-1]}\n\n")

# ImgTeorPDF

# basic preprocessing

### 1 no preprocessing

In [16]:
def preprocess(txt):
    txt =  str(txt)
    terms = word_tokenize(txt)
    
    return terms

In [17]:
data_preprocess = [preprocess(doc) for doc in data]

In [18]:
a = time.time()

In [19]:
bm25 = BM25L(data_preprocess)

In [20]:
d = time.time()

In [21]:
print("Duração indexação: %f" % (d-a))

Duração indexação: 50.077694


### Recall

In [22]:
avaliacaoRecall(5)

KeyboardInterrupt: 

In [ ]:
avaliacaoRecall(10)

In [23]:
avaliacaoRecall(20)

Duração: 139.748135
Encontrados: 194
Total: 295
Acurácia 20 docs: 0.6576271186440678




### Precision

In [93]:
avaliacaoPrecision(5)

Precision: 0.105763


In [94]:
avaliacaoPrecision(10)

Precision: 0.057288


In [95]:
avaliacaoPrecision(20)

Precision: 0.031525


### MAP

In [96]:
avaliacaoMAP(5)

MAP: 0.003404


In [97]:
avaliacaoMAP(10)

MAP: 0.001890


In [98]:
avaliacaoMAP(20)

MAP: 0.001037


### RR

In [99]:
avaliacaoRR(5)

RR: 0.352542


In [25]:
data_preprocess = None

### 5 lowercase + punctuation, acetuation, and stopword removal

In [24]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [word for word in terms if word not in stopwords]

    return terms

In [25]:
data_preprocess = Parallel(n_jobs=4)(delayed(preprocess)(doc) for doc in data)

In [26]:
a = time.time()

In [27]:
bm25 = BM25L(data_preprocess)

In [28]:
d = time.time()

In [29]:
print("Duração indexação: %f" % (d-a))

Duração indexação: 32.583014


### Recall

In [76]:
avaliacaoRecall(5)

Encontrados: 165
Total: 295
Acurácia 20 docs: 0.559322033898305




In [77]:
avaliacaoRecall(10)

Encontrados: 183
Total: 295
Acurácia 20 docs: 0.6203389830508474




In [30]:
avaliacaoRecall(20)

Duração: 100.352020
Encontrados: 211
Total: 295
Acurácia 20 docs: 0.7152542372881356




### Precision

In [79]:
avaliacaoPrecision(5)

Precision: 0.111864


In [80]:
avaliacaoPrecision(10)

Precision: 0.062034


In [81]:
avaliacaoPrecision(20)

Precision: 0.033220


### MAP

In [82]:
avaliacaoMAP(5)

MAP: 0.004413


In [83]:
avaliacaoMAP(10)

MAP: 0.002800


In [84]:
avaliacaoMAP(20)

MAP: 0.001726


### RR

In [85]:
avaliacaoRR(5)

RR: 0.376271


In [86]:
data_preprocess = None

### 8 lowercase + punctuation, acetuation, and stopword removal + stemming (RSLP)

In [31]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = RSLPStemmer()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
        
    return terms

In [32]:
data_preprocess = Parallel(n_jobs=4)(delayed(preprocess)(doc) for doc in data)

In [33]:
a = time.time()

In [34]:
bm25 = BM25L(data_preprocess)

In [35]:
d = time.time()

In [36]:
print("Duração indexação: %f" % (d-a))

Duração indexação: 30.989005


### Recall

In [62]:
avaliacaoRecall(5)

Encontrados: 164
Total: 295
Acurácia 20 docs: 0.5559322033898305




In [63]:
avaliacaoRecall(10)

Encontrados: 184
Total: 295
Acurácia 20 docs: 0.6237288135593221




In [37]:
avaliacaoRecall(20)

Duração: 112.342522
Encontrados: 213
Total: 295
Acurácia 20 docs: 0.7220338983050848




### Precision

In [65]:
avaliacaoPrecision(5)

Precision: 0.111186


In [66]:
avaliacaoPrecision(10)

Precision: 0.062373


In [67]:
avaliacaoPrecision(20)

Precision: 0.034068


### MAP

In [68]:
avaliacaoMAP(5)

MAP: 0.002945


In [69]:
avaliacaoMAP(10)

MAP: 0.002539


In [70]:
avaliacaoMAP(20)

MAP: 0.001766


### RR

In [71]:
avaliacaoRR(5)

RR: 0.376271


In [72]:
data_preprocess = None

### 9 lowercase + punctuation, acetuation, and stopword removal + stemming (Savoy)

In [38]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = Savoy()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
        
    return terms

In [39]:
data_preprocess = Parallel(n_jobs=8)(delayed(preprocess)(doc) for doc in data)

In [40]:
a = time.time()

In [41]:
bm25 = BM25L(data_preprocess)

In [42]:
d = time.time()

In [43]:
print("Duração indexação: %f" % (d-a))

Duração indexação: 38.028011


### Recall

In [48]:
avaliacaoRecall(5)

Encontrados: 169
Total: 295
Acurácia 20 docs: 0.5728813559322034




In [49]:
avaliacaoRecall(10)

Encontrados: 180
Total: 295
Acurácia 20 docs: 0.6101694915254238




In [44]:
avaliacaoRecall(20)

Duração: 111.536998
Encontrados: 213
Total: 295
Acurácia 20 docs: 0.7220338983050848




### Precision

In [51]:
avaliacaoPrecision(5)

Precision: 0.114576


In [52]:
avaliacaoPrecision(10)

Precision: 0.061017


In [53]:
avaliacaoPrecision(20)

Precision: 0.033390


### MAP

In [54]:
avaliacaoMAP(5)

MAP: 0.003035


In [55]:
avaliacaoMAP(10)

MAP: 0.001963


In [56]:
avaliacaoMAP(20)

MAP: 0.001294


### RR

In [57]:
avaliacaoRR(5)

RR: 0.376271


In [58]:
data_preprocess = None

### 18 lowercase + punctuation, acetuation, and stopword removal + stemming (RSLP) + unigram and bigram 

In [45]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = RSLPStemmer()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
    
    ngram = []
    ngram_1 = list(ngrams(terms, 1))
    ngram_2 = list(ngrams(terms, 2))
    for w in ngram_1:
        ngram.append(w[0])
        
    for w in ngram_2:
        string = w[0] + " " + w[1]
        ngram.append(string)
    
    return ngram


In [46]:
data_preprocess = Parallel(n_jobs=8)(delayed(preprocess)(doc) for doc in data)

In [47]:
a = time.time()

In [48]:
bm25 = BM25L(data_preprocess)

In [49]:
d = time.time()

In [50]:
print("Duração indexação: %f" % (d-a))

Duração indexação: 96.374999


### Recall

In [34]:
avaliacaoRecall(5)

Encontrados: 168
Total: 295
Acurácia 20 docs: 0.5694915254237288




In [35]:
avaliacaoRecall(10)

Encontrados: 180
Total: 295
Acurácia 20 docs: 0.6101694915254238




In [51]:
avaliacaoRecall(20)

Duração: 132.835017
Encontrados: 217
Total: 295
Acurácia 20 docs: 0.735593220338983




### Precision

In [37]:
avaliacaoPrecision(5)

Precision: 0.113898


In [38]:
avaliacaoPrecision(10)

Precision: 0.061017


In [39]:
avaliacaoPrecision(20)

Precision: 0.033559


### MAP

In [40]:
avaliacaoMAP(5)

MAP: 0.003007


In [41]:
avaliacaoMAP(10)

MAP: 0.001748


In [42]:
avaliacaoMAP(20)

MAP: 0.001063


### RR

In [43]:
avaliacaoRR(5)

RR: 0.393220


In [44]:
data_preprocess = None

### 21 lowercase + punctuation, acetuation, and stopword removal + stemming (Savoy) + unigram and bigram

In [16]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))
    stemmer = Savoy()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
    
    ngram = []
    ngram_1 = list(ngrams(terms, 1))
    ngram_2 = list(ngrams(terms, 2))
    for w in ngram_1:
        ngram.append(w[0])
        
    for w in ngram_2:
        string = w[0] + " " + w[1]
        ngram.append(string)
    
    return ngram

In [17]:
data_preprocess = Parallel(n_jobs=8)(delayed(preprocess)(doc) for doc in data)

In [18]:
a = time.time()

In [19]:
bm25 = BM25L(data_preprocess)

In [20]:
d = time.time()

In [21]:
print("Duração indexação: %f" % (d-a))

Duração indexação: 116.526019


### Recall

In [21]:
avaliacaoRecall(5)

Encontrados: 169
Total: 295
Acurácia 20 docs: 0.5728813559322034




In [22]:
avaliacaoRecall(10)

Encontrados: 184
Total: 295
Acurácia 20 docs: 0.6237288135593221




In [22]:
avaliacaoRecall(20)

Duração: 133.527922
Encontrados: 211
Total: 295
Acurácia 20 docs: 0.7152542372881356




### Precision

In [24]:
avaliacaoPrecision(5)

Precision: 0.114576


In [25]:
avaliacaoPrecision(10)

Precision: 0.062373


In [26]:
avaliacaoPrecision(20)

Precision: 0.033559


### MAP

In [27]:
avaliacaoMAP(5)

MAP: 0.002960


In [28]:
avaliacaoMAP(10)

MAP: 0.002001


In [29]:
avaliacaoMAP(20)

MAP: 0.001084


### RR

In [30]:
avaliacaoRR(5)

RR: 0.389831


### 22 lowercase + punctuation, acetuation, and stopword removal + stemming (RSLP_S)

In [59]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = RSLP_S()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
        
    return terms

In [60]:
data_preprocess = [preprocess(doc) for doc in data]

In [61]:
a = time.time()

In [62]:
bm25 = BM25L(data_preprocess)

In [63]:
d = time.time()

In [64]:
print("Duração indexação: %f" % (d-a))

Duração indexação: 34.496997


### Recall

In [132]:
avaliacaoRecall(5)

Encontrados: 164
Total: 295
Acurácia 20 docs: 0.5559322033898305




In [133]:
avaliacaoRecall(10)

Encontrados: 178
Total: 295
Acurácia 20 docs: 0.6033898305084746




In [65]:
avaliacaoRecall(20)

Duração: 108.213991
Encontrados: 208
Total: 295
Acurácia 20 docs: 0.7050847457627119




### Precision

In [135]:
avaliacaoPrecision(5)

Precision: 0.111186


In [136]:
avaliacaoPrecision(10)

Precision: 0.060339


In [137]:
avaliacaoPrecision(20)

Precision: 0.032712


### MAP

In [138]:
avaliacaoMAP(5)

MAP: 0.002970


In [139]:
avaliacaoMAP(10)

MAP: 0.001884


In [140]:
avaliacaoMAP(20)

MAP: 0.001251


### RR

In [141]:
avaliacaoRR(5)

RR: 0.372881


In [62]:
data_preprocess = None

### 23 lowercase + punctuation, acetuation, and stopword removal + stemming (RSLP) + unigram and bigram 

In [66]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = RSLP_S()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
    
    ngram = []
    ngram_1 = list(ngrams(terms, 1))
    ngram_2 = list(ngrams(terms, 2))
    for w in ngram_1:
        ngram.append(w[0])
        
    for w in ngram_2:
        string = w[0] + " " + w[1]
        ngram.append(string)
    
    return ngram


In [67]:
data_preprocess = [preprocess(doc) for doc in data]

In [68]:
a = time.time()

In [69]:
bm25 = BM25L(data_preprocess)

In [70]:
d = time.time()

In [71]:
print("Duração indexação: %f" % (d-a))

Duração indexação: 106.779001


### Recall

In [146]:
avaliacaoRecall(5)

Encontrados: 164
Total: 295
Acurácia 20 docs: 0.5559322033898305




In [147]:
avaliacaoRecall(10)

Encontrados: 180
Total: 295
Acurácia 20 docs: 0.6101694915254238




In [72]:
avaliacaoRecall(20)

Duração: 139.398033
Encontrados: 207
Total: 295
Acurácia 20 docs: 0.7016949152542373




### Precision

In [149]:
avaliacaoPrecision(5)

Precision: 0.111186


In [150]:
avaliacaoPrecision(10)

Precision: 0.061017


In [151]:
avaliacaoPrecision(20)

Precision: 0.032881


### MAP

In [152]:
avaliacaoMAP(5)

MAP: 0.002778


In [153]:
avaliacaoMAP(10)

MAP: 0.001897


In [154]:
avaliacaoMAP(20)

MAP: 0.001022


### RR

In [155]:
avaliacaoRR(5)

RR: 0.389831


In [73]:
data_preprocess = None

# Emenda

In [74]:
data = base.txtEmenta.dropna()        # Trocar com nome da coluna desejada
data = data.to_numpy()

In [75]:
def get_name(base, doc):
    return str(base[base.txtEmenta==doc].txtNome.to_numpy()[0]).strip()

In [72]:
def avaliacaoRR(top_k=20):
    primeira_posicao=0
    
    for l,x in zip(y,X):
        tokenized_query3 = preprocess(x)                   
     
        top_n_stem_l = bm25.get_top_n(tokenized_query3,data, n=top_k)
        labelsnsL = [get_name(base,d) for d in top_n_stem_l]
        if str(labelsnsL[0]).strip() == l.strip():
            primeira_posicao+=1
    rr = primeira_posicao / 295
    print("RR: %f" % (rr))

In [22]:
def avaliacaoMAP(top_k=20):
    
    l_v = list()
    labels_nsL = list()
    for l,x in zip(y,X):
        tokenized_query3 = preprocess(x)                   
        top_n_stem_l = bm25.get_top_n(tokenized_query3,data, n=top_k)
        labelsnsL = [get_name(base,d) for d in top_n_stem_l] 
        
        l_v.append(l)
        labels_nsL.append(labelsnsL)
    
    denominador = 0
    
    encontrou=False
    soma=0
    for Y,x in zip(l_v,labels_nsL):
        calc = 0
        quant=0
        for k in x:
            if str(k).strip() == Y.strip():
                quant=1
                encontrou=True
            denominador+=1
            calc=quant/denominador
            if encontrou:
                calc = calc * 1
            else:
                calc = calc * 0
            soma+=calc
            encontrou = False
    MAP = soma /295
    print("MAP: %f" % (MAP))            

In [23]:
def avaliacaoPrecision(top_k=20):
    l_v = list()
    labels_nsL = list()
    for l,x in zip(y,X):
        tokenized_query3 = preprocess(x)                   
        top_n_stem_l = bm25.get_top_n(tokenized_query3,data, n=top_k)
        labelsnsL = [get_name(base,d) for d in top_n_stem_l] 
        
        l_v.append(l)
        labels_nsL.append(labelsnsL)
    
    soma=0
    for Y,x in zip(l_v,labels_nsL):
        calc = 0
        for k in x:
            if str(k).strip() == Y.strip():
                calc=1
                break
        calc=calc/top_k
        soma+=calc
    precision = soma /295
    print("Precision: %f" % (precision))

In [24]:
def avaliacaoRecall(top_k=20):

    labels_nsL = list()


    l_v = list()
    for l,x in zip(y,X):

        tokenized_query3 = preprocess(x)                   
     
        top_n_stem_l = bm25.get_top_n(tokenized_query3,data, n=top_k)
        
   
        labelsnsL = [get_name(base,d) for d in top_n_stem_l]              #L
 
        labels_nsL.append(labelsnsL)    #L
        
    
        l_v.append(l)
        
    
    res_nsL = list()                #Stopwords + Acentuação + Stemming + Unigram + Bigram (L)


    for k in range(1, top_k+1):


        resultsnsL = list()         #Stopwords + Acentuação + Stemming + Unigram + Bigram (L)


        for i in range(len(l_v)):


            if (l_v[i] in labels_nsL[i][:k]):
                resultsnsL.append(1)
            else:
                resultsnsL.append(0)

       
        res_nsL.append(sum(resultsnsL)/len(resultsnsL))
    print(f"Encontrados: {sum(resultsnsL)}")
    print(f"Total: {len(arr_assunto)}")
#     print(f"Acurácia 10 docs: {res_nsL[9]}")
    print(f"Acurácia 20 docs: {res_nsL[-1]}\n\n")

# basic preprocessing

### 1 no preprocessing

In [76]:
def preprocess(txt):
    txt =  str(txt)
    terms = word_tokenize(txt)
    
    return terms

In [77]:
data_preprocess = [preprocess(doc) for doc in data]

In [78]:
a = time.time()

In [79]:
bm25 = BM25L(data_preprocess)

In [80]:
d = time.time()

In [81]:
print("Duração indexação: %f" % (d-a))

Duração indexação: 5.655980


### Recall

In [30]:
avaliacaoRecall(5)

Encontrados: 86
Total: 295
Acurácia 20 docs: 0.29152542372881357




In [31]:
avaliacaoRecall(10)

Encontrados: 95
Total: 295
Acurácia 20 docs: 0.3220338983050847




In [82]:
avaliacaoRecall(20)

Duração: 130.274996
Encontrados: 116
Total: 295
Acurácia 20 docs: 0.39322033898305087




### Precision

In [33]:
avaliacaoPrecision(5)

Precision: 0.058305


In [34]:
avaliacaoPrecision(10)

Precision: 0.032203


In [35]:
avaliacaoPrecision(20)

Precision: 0.017966


### MAP

In [36]:
avaliacaoMAP(5)

MAP: 0.001960


In [37]:
avaliacaoMAP(10)

MAP: 0.001133


In [38]:
avaliacaoMAP(20)

MAP: 0.000614


### RR

In [40]:
avaliacaoRR(5)

RR: 0.200000


In [83]:
data_preprocess = None

### 5 lowercase + punctuation, acetuation, and stopword removal

In [84]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [word for word in terms if word not in stopwords]

    return terms

In [85]:
data_preprocess = [preprocess(doc) for doc in data]

In [86]:
a = time.time()

In [87]:
bm25 = BM25L(data_preprocess)

In [88]:
d = time.time()

In [89]:
print("Duração indexação: %f" % (d-a))

Duração indexação: 1.163001


### Recall

In [46]:
avaliacaoRecall(5)

Encontrados: 110
Total: 295
Acurácia 20 docs: 0.3728813559322034




In [47]:
avaliacaoRecall(10)

Encontrados: 122
Total: 295
Acurácia 20 docs: 0.4135593220338983




In [90]:
avaliacaoRecall(20)

Duração: 80.026951
Encontrados: 139
Total: 295
Acurácia 20 docs: 0.4711864406779661




### Precision

In [49]:
avaliacaoPrecision(5)

Precision: 0.074576


In [50]:
avaliacaoPrecision(10)

Precision: 0.041356


In [51]:
avaliacaoPrecision(20)

Precision: 0.022373


### MAP

In [52]:
avaliacaoMAP(5)

MAP: 0.002433


In [53]:
avaliacaoMAP(10)

MAP: 0.001301


In [54]:
avaliacaoMAP(20)

MAP: 0.000704


### RR

In [55]:
avaliacaoRR(5)

RR: 0.237288


In [85]:
data_preprocess = None

### 8 lowercase + punctuation, acetuation, and stopword removal + stemming (RSLP)

In [91]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = RSLPStemmer()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
        
    return terms

In [92]:
data_preprocess = [preprocess(doc) for doc in data]

In [93]:
a = time.time()

In [94]:
bm25 = BM25L(data_preprocess)

In [95]:
d = time.time()

In [96]:
print("Duração indexação: %f" % (d-a))

Duração indexação: 1.079230


### Recall

In [62]:
avaliacaoRecall(5)

Encontrados: 120
Total: 295
Acurácia 20 docs: 0.4067796610169492




In [63]:
avaliacaoRecall(10)

Encontrados: 134
Total: 295
Acurácia 20 docs: 0.4542372881355932




In [97]:
avaliacaoRecall(20)

Duração: 81.812033
Encontrados: 151
Total: 295
Acurácia 20 docs: 0.511864406779661




### Precision

In [65]:
avaliacaoPrecision(5)

Precision: 0.081356


In [66]:
avaliacaoPrecision(10)

Precision: 0.045424


In [67]:
avaliacaoPrecision(20)

Precision: 0.024746


### MAP

In [68]:
avaliacaoMAP(5)

MAP: 0.002689


In [69]:
avaliacaoMAP(10)

MAP: 0.001449


In [70]:
avaliacaoMAP(20)

MAP: 0.000786


### RR

In [71]:
avaliacaoRR(5)

RR: 0.244068


In [98]:
data_preprocess = None

### 9 lowercase + punctuation, acetuation, and stopword removal + stemming (Savoy)

In [99]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = Savoy()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
        
    return terms

In [100]:
data_preprocess = [preprocess(doc) for doc in data]

In [101]:
a = time.time()

In [102]:
bm25 = BM25L(data_preprocess)

In [103]:
d = time.time()

In [104]:
print("Duração indexação: %f" % (d-a))

Duração indexação: 1.075005


### Recall

In [78]:
avaliacaoRecall(5)

Encontrados: 121
Total: 295
Acurácia 20 docs: 0.4101694915254237




In [79]:
avaliacaoRecall(10)

Encontrados: 133
Total: 295
Acurácia 20 docs: 0.45084745762711864




In [105]:
avaliacaoRecall(20)

Duração: 82.225000
Encontrados: 151
Total: 295
Acurácia 20 docs: 0.511864406779661




### Precision

In [81]:
avaliacaoPrecision(5)

Precision: 0.082034


In [82]:
avaliacaoPrecision(10)

Precision: 0.045085


In [83]:
avaliacaoPrecision(20)

Precision: 0.024576


### MAP

In [84]:
avaliacaoMAP(5)

MAP: 0.002659


In [85]:
avaliacaoMAP(10)

MAP: 0.001446


In [86]:
avaliacaoMAP(20)

MAP: 0.000772


### RR

In [87]:
avaliacaoRR(5)

RR: 0.254237


In [98]:
data_preprocess = None

### 18 lowercase + punctuation, acetuation, and stopword removal + stemming (RSLP) + unigram and bigram 

In [106]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = RSLPStemmer()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
    
    ngram = []
    ngram_1 = list(ngrams(terms, 1))
    ngram_2 = list(ngrams(terms, 2))
    for w in ngram_1:
        ngram.append(w[0])
        
    for w in ngram_2:
        string = w[0] + " " + w[1]
        ngram.append(string)
    
    return ngram


In [107]:
data_preprocess = [preprocess(doc) for doc in data]

In [108]:
a = time.time()

In [109]:
bm25 = BM25L(data_preprocess)

In [110]:
d = time.time()

In [111]:
print("Duração indexação: %f" % (d-a))

Duração indexação: 2.639907


### Recall

In [94]:
avaliacaoRecall(5)

Encontrados: 120
Total: 295
Acurácia 20 docs: 0.4067796610169492




In [95]:
avaliacaoRecall(10)

Encontrados: 135
Total: 295
Acurácia 20 docs: 0.4576271186440678




In [112]:
avaliacaoRecall(20)

Duração: 106.037029
Encontrados: 150
Total: 295
Acurácia 20 docs: 0.5084745762711864




### Precision

In [97]:
avaliacaoPrecision(5)

Precision: 0.081356


In [98]:
avaliacaoPrecision(10)

Precision: 0.045763


In [99]:
avaliacaoPrecision(20)

Precision: 0.025085


### MAP

In [100]:
avaliacaoMAP(5)

MAP: 0.002609


In [101]:
avaliacaoMAP(10)

MAP: 0.001431


In [102]:
avaliacaoMAP(20)

MAP: 0.000786


### RR

In [103]:
avaliacaoRR(5)

RR: 0.257627


In [106]:
data_preprocess = None

### 21 lowercase + punctuation, acetuation, and stopword removal + stemming (Savoy) + unigram and bigram

In [113]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))
    stemmer = Savoy()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
    
    ngram = []
    ngram_1 = list(ngrams(terms, 1))
    ngram_2 = list(ngrams(terms, 2))
    for w in ngram_1:
        ngram.append(w[0])
        
    for w in ngram_2:
        string = w[0] + " " + w[1]
        ngram.append(string)
    
    return ngram

In [114]:
data_preprocess = [preprocess(doc) for doc in data]

In [115]:
a = time.time()

In [116]:
bm25 = BM25L(data_preprocess)

In [117]:
d = time.time()

In [118]:
print("Duração indexação: %f" % (d-a))

Duração indexação: 4.980199


### Recall

In [110]:
avaliacaoRecall(5)

Encontrados: 119
Total: 295
Acurácia 20 docs: 0.4033898305084746




In [111]:
avaliacaoRecall(10)

Encontrados: 135
Total: 295
Acurácia 20 docs: 0.4576271186440678




In [119]:
avaliacaoRecall(20)

Duração: 103.243001
Encontrados: 151
Total: 295
Acurácia 20 docs: 0.511864406779661




### Precision

In [113]:
avaliacaoPrecision(5)

Precision: 0.080678


In [114]:
avaliacaoPrecision(10)

Precision: 0.045763


In [115]:
avaliacaoPrecision(20)

Precision: 0.024407


### MAP

In [22]:
avaliacaoMAP(5)

MAP: 0.002598


In [23]:
avaliacaoMAP(10)

MAP: 0.001439


In [24]:
avaliacaoMAP(20)

MAP: 0.000775


### RR

In [25]:
avaliacaoRR(5)

RR: 0.254237


### 22 lowercase + punctuation, acetuation, and stopword removal + stemming (RSLP_S)

In [120]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = RSLP_S()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
        
    return terms

In [121]:
data_preprocess = [preprocess(doc) for doc in data]

In [122]:
a = time.time()

In [123]:
bm25 = BM25L(data_preprocess)

In [124]:
d = time.time()

In [125]:
print("Duração indexação: %f" % (d-a))

Duração indexação: 1.198999


### Recall

In [30]:
avaliacaoRecall(5)

Encontrados: 118
Total: 295
Acurácia 20 docs: 0.4




In [31]:
avaliacaoRecall(10)

Encontrados: 127
Total: 295
Acurácia 20 docs: 0.43050847457627117




In [126]:
avaliacaoRecall(20)

Duração: 78.250034
Encontrados: 148
Total: 295
Acurácia 20 docs: 0.5016949152542373




### Precision

In [33]:
avaliacaoPrecision(5)

Precision: 0.080000


In [34]:
avaliacaoPrecision(10)

Precision: 0.043051


In [35]:
avaliacaoPrecision(20)

Precision: 0.023559


### MAP

In [36]:
avaliacaoMAP(5)

MAP: 0.002493


In [37]:
avaliacaoMAP(10)

MAP: 0.001350


In [38]:
avaliacaoMAP(20)

MAP: 0.000755


### RR

In [39]:
avaliacaoRR(5)

RR: 0.237288


In [127]:
data_preprocess = None

### 23 lowercase + punctuation, acetuation, and stopword removal + stemming (RSLP) + unigram and bigram 

In [128]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = RSLP_S()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
    
    ngram = []
    ngram_1 = list(ngrams(terms, 1))
    ngram_2 = list(ngrams(terms, 2))
    for w in ngram_1:
        ngram.append(w[0])
        
    for w in ngram_2:
        string = w[0] + " " + w[1]
        ngram.append(string)
    
    return ngram


In [129]:
data_preprocess = [preprocess(doc) for doc in data]

In [130]:
a = time.time()

In [131]:
bm25 = BM25L(data_preprocess)

In [132]:
d = time.time()

In [133]:
print("Duração indexação: %f" % (d-a))

Duração indexação: 3.145536


### Recall

In [44]:
avaliacaoRecall(5)

Encontrados: 116
Total: 295
Acurácia 20 docs: 0.39322033898305087




In [45]:
avaliacaoRecall(10)

Encontrados: 130
Total: 295
Acurácia 20 docs: 0.4406779661016949




In [134]:
avaliacaoRecall(20)

Duração: 98.935030
Encontrados: 147
Total: 295
Acurácia 20 docs: 0.49830508474576274




### Precision

In [47]:
avaliacaoPrecision(5)

Precision: 0.078644


In [48]:
avaliacaoPrecision(10)

Precision: 0.044068


In [49]:
avaliacaoPrecision(20)

Precision: 0.023898


### MAP

In [50]:
avaliacaoMAP(5)

MAP: 0.002432


In [51]:
avaliacaoMAP(10)

MAP: 0.001368


In [52]:
avaliacaoMAP(20)

MAP: 0.000763


### RR

In [53]:
avaliacaoRR(5)

RR: 0.257627


In [135]:
data_preprocess = None

# Indexacao

In [136]:
data = base.txtIndexacao.dropna()        # Trocar com nome da coluna desejada
data = data.to_numpy()

In [137]:
def get_name(base, doc):
    return str(base[base.txtIndexacao==doc].txtNome.to_numpy()[0]).strip()

In [138]:
def avaliacaoRR(top_k=20):
    primeira_posicao=0
    
    for l,x in zip(y,X):
        tokenized_query3 = preprocess(x)                   
     
        top_n_stem_l = bm25.get_top_n(tokenized_query3,data, n=top_k)
        labelsnsL = [get_name(base,d) for d in top_n_stem_l]
        if str(labelsnsL[0]).strip() == l.strip():
            primeira_posicao+=1
    rr = primeira_posicao / 295
    print("RR: %f" % (rr))

In [57]:
def avaliacaoMAP(top_k=20):
    
    l_v = list()
    labels_nsL = list()
    for l,x in zip(y,X):
        tokenized_query3 = preprocess(x)                   
        top_n_stem_l = bm25.get_top_n(tokenized_query3,data, n=top_k)
        labelsnsL = [get_name(base,d) for d in top_n_stem_l] 
        
        l_v.append(l)
        labels_nsL.append(labelsnsL)
    
    denominador = 0
    
    encontrou=False
    soma=0
    for Y,x in zip(l_v,labels_nsL):
        calc = 0
        quant=0
        for k in x:
            if str(k).strip() == Y.strip():
                quant=1
                encontrou=True
            denominador+=1
            calc=quant/denominador
            if encontrou:
                calc = calc * 1
            else:
                calc = calc * 0
            soma+=calc
            encontrou = False
    MAP = soma /295
    print("MAP: %f" % (MAP))            

In [58]:
def avaliacaoPrecision(top_k=20):
    l_v = list()
    labels_nsL = list()
    for l,x in zip(y,X):
        tokenized_query3 = preprocess(x)                   
        top_n_stem_l = bm25.get_top_n(tokenized_query3,data, n=top_k)
        labelsnsL = [get_name(base,d) for d in top_n_stem_l] 
        
        l_v.append(l)
        labels_nsL.append(labelsnsL)
    
    soma=0
    for Y,x in zip(l_v,labels_nsL):
        calc = 0
        for k in x:
            if str(k).strip() == Y.strip():
                calc=1
                break
        calc=calc/top_k
        soma+=calc
    precision = soma /295
    print("Precision: %f" % (precision))

In [59]:
def avaliacaoRecall(top_k=20):

    labels_nsL = list()


    l_v = list()
    for l,x in zip(y,X):

        tokenized_query3 = preprocess(x)                   
     
        top_n_stem_l = bm25.get_top_n(tokenized_query3,data, n=top_k)
        
   
        labelsnsL = [get_name(base,d) for d in top_n_stem_l]              #L
 
        labels_nsL.append(labelsnsL)    #L
        
    
        l_v.append(l)
        
    
    res_nsL = list()                #Stopwords + Acentuação + Stemming + Unigram + Bigram (L)


    for k in range(1, top_k+1):


        resultsnsL = list()         #Stopwords + Acentuação + Stemming + Unigram + Bigram (L)


        for i in range(len(l_v)):


            if (l_v[i] in labels_nsL[i][:k]):
                resultsnsL.append(1)
            else:
                resultsnsL.append(0)

       
        res_nsL.append(sum(resultsnsL)/len(resultsnsL))
    print(f"Encontrados: {sum(resultsnsL)}")
    print(f"Total: {len(arr_assunto)}")
#     print(f"Acurácia 10 docs: {res_nsL[9]}")
    print(f"Acurácia 20 docs: {res_nsL[-1]}\n\n")

# basic preprocessing

### 1 no preprocessing

In [139]:
def preprocess(txt):
    txt =  str(txt)
    terms = word_tokenize(txt)
    
    return terms

In [140]:
data_preprocess = [preprocess(doc) for doc in data]

In [141]:
a = time.time()

In [142]:
bm25 = BM25L(data_preprocess)

In [143]:
d = time.time()

In [144]:
print("Duração indexação: %f" % (d-a))

Duração indexação: 1.876001


### Recall

In [35]:
avaliacaoRecall(5)

Encontrados: 39
Total: 295
Acurácia 20 docs: 0.13220338983050847




In [36]:
avaliacaoRecall(10)

Encontrados: 45
Total: 295
Acurácia 20 docs: 0.15254237288135594




In [145]:
avaliacaoRecall(20)

Duração: 106.977000
Encontrados: 69
Total: 295
Acurácia 20 docs: 0.23389830508474577




### Precision

In [38]:
avaliacaoPrecision(5)

Precision: 0.026441


In [39]:
avaliacaoPrecision(10)

Precision: 0.015254


In [40]:
avaliacaoPrecision(20)

Precision: 0.009153


### MAP

In [41]:
avaliacaoMAP(5)

MAP: 0.000321


In [42]:
avaliacaoMAP(10)

MAP: 0.000481


In [43]:
avaliacaoMAP(20)

MAP: 0.000293


### RR

In [44]:
avaliacaoRR(5)

RR: 0.064407


In [146]:
data_preprocess = None

### 5 lowercase + punctuation, acetuation, and stopword removal

In [147]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [word for word in terms if word not in stopwords]

    return terms

In [148]:
data_preprocess = [preprocess(doc) for doc in data]

In [149]:
a = time.time()

In [150]:
bm25 = BM25L(data_preprocess)

In [151]:
d = time.time()

In [152]:
print("Duração indexação: %f" % (d-a))

Duração indexação: 1.199003


### Recall

In [49]:
avaliacaoRecall(5)

Encontrados: 90
Total: 295
Acurácia 20 docs: 0.3050847457627119




In [50]:
avaliacaoRecall(10)

Encontrados: 102
Total: 295
Acurácia 20 docs: 0.34576271186440677




In [153]:
avaliacaoRecall(20)

Duração: 65.834028
Encontrados: 117
Total: 295
Acurácia 20 docs: 0.39661016949152544




### Precision

In [52]:
avaliacaoPrecision(5)

Precision: 0.061017


In [53]:
avaliacaoPrecision(10)

Precision: 0.034576


In [54]:
avaliacaoPrecision(20)

Precision: 0.019492


### MAP

In [55]:
avaliacaoMAP(5)

MAP: 0.001418


In [56]:
avaliacaoMAP(10)

MAP: 0.000926


In [57]:
avaliacaoMAP(20)

MAP: 0.000533


### RR

In [58]:
avaliacaoRR(5)

RR: 0.166102


In [154]:
data_preprocess = None

### 8 lowercase + punctuation, acetuation, and stopword removal + stemming (RSLP)

In [155]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = RSLPStemmer()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
        
    return terms

In [156]:
data_preprocess = [preprocess(doc) for doc in data]

In [157]:
a = time.time()

In [158]:
bm25 = BM25L(data_preprocess)

In [159]:
d = time.time()

In [160]:
print("Duração indexação: %f" % (d-a))

Duração indexação: 1.167982


### Recall

In [63]:
avaliacaoRecall(5)

Encontrados: 111
Total: 295
Acurácia 20 docs: 0.376271186440678




In [64]:
avaliacaoRecall(10)

Encontrados: 126
Total: 295
Acurácia 20 docs: 0.4271186440677966




In [161]:
avaliacaoRecall(20)

Duração: 77.003001
Encontrados: 148
Total: 295
Acurácia 20 docs: 0.5016949152542373




### Precision

In [66]:
avaliacaoPrecision(5)

Precision: 0.075254


In [67]:
avaliacaoPrecision(10)

Precision: 0.042712


In [68]:
avaliacaoPrecision(20)

Precision: 0.023898


### MAP

In [69]:
avaliacaoMAP(5)

MAP: 0.001646


In [70]:
avaliacaoMAP(10)

MAP: 0.001165


In [71]:
avaliacaoMAP(20)

MAP: 0.000665


### RR

In [72]:
avaliacaoRR(5)

RR: 0.240678


In [162]:
data_preprocess = None

### 9 lowercase + punctuation, acetuation, and stopword removal + stemming (Savoy)

In [163]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = Savoy()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
        
    return terms

In [164]:
data_preprocess = [preprocess(doc) for doc in data]

In [165]:
a = time.time()

In [166]:
bm25 = BM25L(data_preprocess)

In [167]:
d = time.time()

In [168]:
print("Duração indexação: %f" % (d-a))

Duração indexação: 1.714000


### Recall

In [77]:
avaliacaoRecall(5)

Encontrados: 116
Total: 295
Acurácia 20 docs: 0.39322033898305087




In [78]:
avaliacaoRecall(10)

Encontrados: 129
Total: 295
Acurácia 20 docs: 0.43728813559322033




In [169]:
avaliacaoRecall(20)

Duração: 70.536997
Encontrados: 149
Total: 295
Acurácia 20 docs: 0.5050847457627119




### Precision

In [80]:
avaliacaoPrecision(5)

Precision: 0.078644


In [81]:
avaliacaoPrecision(10)

Precision: 0.043729


In [82]:
avaliacaoPrecision(20)

Precision: 0.024237


### MAP

In [83]:
avaliacaoMAP(5)

MAP: 0.001542


In [84]:
avaliacaoMAP(10)

MAP: 0.001150


In [85]:
avaliacaoMAP(20)

MAP: 0.000678


### RR

In [86]:
avaliacaoRR(5)

RR: 0.240678


In [170]:
data_preprocess = None

### 18 lowercase + punctuation, acetuation, and stopword removal + stemming (RSLP) + unigram and bigram 

In [171]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = RSLPStemmer()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
    
    ngram = []
    ngram_1 = list(ngrams(terms, 1))
    ngram_2 = list(ngrams(terms, 2))
    for w in ngram_1:
        ngram.append(w[0])
        
    for w in ngram_2:
        string = w[0] + " " + w[1]
        ngram.append(string)
    
    return ngram


In [172]:
data_preprocess = [preprocess(doc) for doc in data]

In [173]:
a = time.time()

In [174]:
bm25 = BM25L(data_preprocess)

In [175]:
d = time.time()

In [176]:
print("Duração indexação: %f" % (d-a))

Duração indexação: 3.505993


### Recall

In [91]:
avaliacaoRecall(5)

Encontrados: 108
Total: 295
Acurácia 20 docs: 0.36610169491525424




In [92]:
avaliacaoRecall(10)

Encontrados: 122
Total: 295
Acurácia 20 docs: 0.4135593220338983




In [177]:
avaliacaoRecall(20)

Duração: 91.166031
Encontrados: 145
Total: 295
Acurácia 20 docs: 0.4915254237288136




### Precision

In [94]:
avaliacaoPrecision(5)

Precision: 0.073220


In [95]:
avaliacaoPrecision(10)

Precision: 0.041356


In [96]:
avaliacaoPrecision(20)

Precision: 0.023220


### MAP

In [97]:
avaliacaoMAP(5)

MAP: 0.001479


In [98]:
avaliacaoMAP(10)

MAP: 0.001090


In [99]:
avaliacaoMAP(20)

MAP: 0.000641


### RR

In [100]:
avaliacaoRR(5)

RR: 0.237288


In [178]:
data_preprocess = None

### 21 lowercase + punctuation, acetuation, and stopword removal + stemming (Savoy) + unigram and bigram

In [179]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))
    stemmer = Savoy()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
    
    ngram = []
    ngram_1 = list(ngrams(terms, 1))
    ngram_2 = list(ngrams(terms, 2))
    for w in ngram_1:
        ngram.append(w[0])
        
    for w in ngram_2:
        string = w[0] + " " + w[1]
        ngram.append(string)
    
    return ngram

In [180]:
data_preprocess = [preprocess(doc) for doc in data]

In [181]:
a = time.time()

In [182]:
bm25 = BM25L(data_preprocess)

In [183]:
d = time.time()

In [184]:
print("Duração indexação: %f" % (d-a))

Duração indexação: 4.299007


### Recall

In [105]:
avaliacaoRecall(5)

Encontrados: 107
Total: 295
Acurácia 20 docs: 0.36271186440677966




In [106]:
avaliacaoRecall(10)

Encontrados: 125
Total: 295
Acurácia 20 docs: 0.423728813559322




In [185]:
avaliacaoRecall(20)

Duração: 85.290036
Encontrados: 147
Total: 295
Acurácia 20 docs: 0.49830508474576274




### Precision

In [108]:
avaliacaoPrecision(5)

Precision: 0.072542


In [109]:
avaliacaoPrecision(10)

Precision: 0.042373


In [110]:
avaliacaoPrecision(20)

Precision: 0.023729


### MAP

In [111]:
avaliacaoMAP(5)

MAP: 0.001414


In [112]:
avaliacaoMAP(10)

MAP: 0.000953


In [113]:
avaliacaoMAP(20)

MAP: 0.000640


### RR

In [114]:
avaliacaoRR(5)

RR: 0.216949


### 22 lowercase + punctuation, acetuation, and stopword removal + stemming (RSLP_S)

In [186]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = RSLP_S()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
        
    return terms

In [187]:
data_preprocess = [preprocess(doc) for doc in data]

In [188]:
a = time.time()

In [189]:
bm25 = BM25L(data_preprocess)

In [190]:
d = time.time()

In [191]:
print("Duração indexação: %f" % (d-a))

Duração indexação: 1.359988


### Recall

In [63]:
avaliacaoRecall(5)

Encontrados: 113
Total: 295
Acurácia 20 docs: 0.38305084745762713




In [64]:
avaliacaoRecall(10)

Encontrados: 125
Total: 295
Acurácia 20 docs: 0.423728813559322




In [192]:
avaliacaoRecall(20)

Duração: 70.707999
Encontrados: 148
Total: 295
Acurácia 20 docs: 0.5016949152542373




### Precision

In [66]:
avaliacaoPrecision(5)

Precision: 0.076610


In [67]:
avaliacaoPrecision(10)

Precision: 0.042373


In [68]:
avaliacaoPrecision(20)

Precision: 0.023729


### MAP

In [69]:
avaliacaoMAP(5)

MAP: 0.001513


In [70]:
avaliacaoMAP(10)

MAP: 0.001092


In [71]:
avaliacaoMAP(20)

MAP: 0.000630


### RR

In [72]:
avaliacaoRR(5)

RR: 0.237288


In [193]:
data_preprocess = None

### 23 lowercase + punctuation, acetuation, and stopword removal + stemming (RSLP) + unigram and bigram 

In [194]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = RSLP_S()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
    
    ngram = []
    ngram_1 = list(ngrams(terms, 1))
    ngram_2 = list(ngrams(terms, 2))
    for w in ngram_1:
        ngram.append(w[0])
        
    for w in ngram_2:
        string = w[0] + " " + w[1]
        ngram.append(string)
    
    return ngram


In [195]:
data_preprocess = [preprocess(doc) for doc in data]

In [196]:
a = time.time()

In [197]:
bm25 = BM25L(data_preprocess)

In [198]:
d = time.time()

In [199]:
print("Duração indexação: %f" % (d-a))

Duração indexação: 3.636016


### Recall

In [77]:
avaliacaoRecall(5)

Encontrados: 103
Total: 295
Acurácia 20 docs: 0.34915254237288135




In [78]:
avaliacaoRecall(10)

Encontrados: 120
Total: 295
Acurácia 20 docs: 0.4067796610169492




In [200]:
avaliacaoRecall(20)

Duração: 81.711001
Encontrados: 142
Total: 295
Acurácia 20 docs: 0.48135593220338985




### Precision

In [80]:
avaliacaoPrecision(5)

Precision: 0.069831


In [81]:
avaliacaoPrecision(10)

Precision: 0.040678


In [82]:
avaliacaoPrecision(20)

Precision: 0.023559


### MAP

In [83]:
avaliacaoMAP(5)

MAP: 0.001260


In [84]:
avaliacaoMAP(10)

MAP: 0.001028


In [85]:
avaliacaoMAP(20)

MAP: 0.000646


### RR

In [86]:
avaliacaoRR(5)

RR: 0.220339


In [201]:
data_preprocess = None

# Emenda + Indexacao

In [202]:
def get_name(base, doc):
    return str(base[base.txtIndexacao==doc].txtNome.to_numpy()[0]).strip()

In [203]:
base['txtIndexacao'] = base['txtEmenta'] + base['txtIndexacao']

In [204]:
data = base.txtIndexacao.dropna()        # Trocar com nome da coluna desejada
data = data.to_numpy()

In [91]:
def avaliacaoRR(top_k=20):
    primeira_posicao=0
    
    for l,x in zip(y,X):
        tokenized_query3 = preprocess(x)                   
     
        top_n_stem_l = bm25.get_top_n(tokenized_query3,data, n=top_k)
        labelsnsL = [get_name(base,d) for d in top_n_stem_l]
        if str(labelsnsL[0]).strip() == l.strip():
            primeira_posicao+=1
    rr = primeira_posicao / 295
    print("RR: %f" % (rr))

In [92]:
def avaliacaoMAP(top_k=20):
    
    l_v = list()
    labels_nsL = list()
    for l,x in zip(y,X):
        tokenized_query3 = preprocess(x)                   
        top_n_stem_l = bm25.get_top_n(tokenized_query3,data, n=top_k)
        labelsnsL = [get_name(base,d) for d in top_n_stem_l] 
        
        l_v.append(l)
        labels_nsL.append(labelsnsL)
    
    denominador = 0
    
    encontrou=False
    soma=0
    for Y,x in zip(l_v,labels_nsL):
        calc = 0
        quant=0
        for k in x:
            if str(k).strip() == Y.strip():
                quant=1
                encontrou=True
            denominador+=1
            calc=quant/denominador
            if encontrou:
                calc = calc * 1
            else:
                calc = calc * 0
            soma+=calc
            encontrou = False
    MAP = soma /295
    print("MAP: %f" % (MAP))            

In [93]:
def avaliacaoPrecision(top_k=20):
    l_v = list()
    labels_nsL = list()
    for l,x in zip(y,X):
        tokenized_query3 = preprocess(x)                   
        top_n_stem_l = bm25.get_top_n(tokenized_query3,data, n=top_k)
        labelsnsL = [get_name(base,d) for d in top_n_stem_l] 
        
        l_v.append(l)
        labels_nsL.append(labelsnsL)
    
    soma=0
    for Y,x in zip(l_v,labels_nsL):
        calc = 0
        for k in x:
            if str(k).strip() == Y.strip():
                calc=1
                break
        calc=calc/top_k
        soma+=calc
    precision = soma /295
    print("Precision: %f" % (precision))

In [94]:
def avaliacaoRecall(top_k=20):

    labels_nsL = list()


    l_v = list()
    for l,x in zip(y,X):

        tokenized_query3 = preprocess(x)                   
     
        top_n_stem_l = bm25.get_top_n(tokenized_query3,data, n=top_k)
        
   
        labelsnsL = [get_name(base,d) for d in top_n_stem_l]              #L
 
        labels_nsL.append(labelsnsL)    #L
        
    
        l_v.append(l)
        
    
    res_nsL = list()                #Stopwords + Acentuação + Stemming + Unigram + Bigram (L)


    for k in range(1, top_k+1):


        resultsnsL = list()         #Stopwords + Acentuação + Stemming + Unigram + Bigram (L)


        for i in range(len(l_v)):


            if (l_v[i] in labels_nsL[i][:k]):
                resultsnsL.append(1)
            else:
                resultsnsL.append(0)

       
        res_nsL.append(sum(resultsnsL)/len(resultsnsL))
    print(f"Encontrados: {sum(resultsnsL)}")
    print(f"Total: {len(arr_assunto)}")
#     print(f"Acurácia 10 docs: {res_nsL[9]}")
    print(f"Acurácia 20 docs: {res_nsL[-1]}\n\n")

# basic preprocessing

### 1 no preprocessing

In [205]:
def preprocess(txt):
    txt =  str(txt)
    terms = word_tokenize(txt)
    
    return terms

In [206]:
data_preprocess = [preprocess(doc) for doc in data]

In [207]:
a = time.time()

In [208]:
bm25 = BM25L(data_preprocess)

In [209]:
d = time.time()

In [210]:
print("Duração indexação: %f" % (d-a))

Duração indexação: 3.337997


### Recall

In [125]:
avaliacaoRecall(5)

Encontrados: 102
Total: 295
Acurácia 20 docs: 0.34576271186440677




In [126]:
avaliacaoRecall(10)

Encontrados: 116
Total: 295
Acurácia 20 docs: 0.39322033898305087




In [211]:
avaliacaoRecall(20)

Duração: 133.149001
Encontrados: 136
Total: 295
Acurácia 20 docs: 0.4610169491525424




### Precision

In [128]:
avaliacaoPrecision(5)

Precision: 0.069153


In [129]:
avaliacaoPrecision(10)

Precision: 0.039322


In [130]:
avaliacaoPrecision(20)

Precision: 0.021356


### MAP

In [131]:
avaliacaoMAP(5)

MAP: 0.002096


In [132]:
avaliacaoMAP(10)

MAP: 0.001223


In [133]:
avaliacaoMAP(20)

MAP: 0.000792


### RR

In [134]:
avaliacaoRR(5)

RR: 0.227119


In [212]:
data_preprocess = None

### 5 lowercase + punctuation, acetuation, and stopword removal

In [213]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [word for word in terms if word not in stopwords]

    return terms

In [214]:
data_preprocess = [preprocess(doc) for doc in data]

In [215]:
a = time.time()

In [216]:
bm25 = BM25L(data_preprocess)

In [217]:
d = time.time()

In [218]:
print("Duração indexação: %f" % (d-a))

Duração indexação: 2.198020


### Recall

In [139]:
avaliacaoRecall(5)

Encontrados: 128
Total: 295
Acurácia 20 docs: 0.43389830508474575




In [140]:
avaliacaoRecall(10)

Encontrados: 142
Total: 295
Acurácia 20 docs: 0.48135593220338985




In [219]:
avaliacaoRecall(20)

Duração: 84.624031
Encontrados: 174
Total: 295
Acurácia 20 docs: 0.5898305084745763




### Precision

In [142]:
avaliacaoPrecision(5)

Precision: 0.086780


In [143]:
avaliacaoPrecision(10)

Precision: 0.048136


In [144]:
avaliacaoPrecision(20)

Precision: 0.027966


### MAP

In [145]:
avaliacaoMAP(5)

MAP: 0.002595


In [146]:
avaliacaoMAP(10)

MAP: 0.001431


In [147]:
avaliacaoMAP(20)

MAP: 0.000913


### RR

In [148]:
avaliacaoRR(5)

RR: 0.274576


In [220]:
data_preprocess = None

### 8 lowercase + punctuation, acetuation, and stopword removal + stemming (RSLP)

In [221]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = RSLPStemmer()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
        
    return terms

In [222]:
data_preprocess = [preprocess(doc) for doc in data]

In [223]:
a = time.time()

In [224]:
bm25 = BM25L(data_preprocess)

In [225]:
d = time.time()

In [226]:
print("Duração indexação: %f" % (d-a))

Duração indexação: 2.019977


### Recall

In [153]:
avaliacaoRecall(5)

Encontrados: 131
Total: 295
Acurácia 20 docs: 0.4440677966101695




In [154]:
avaliacaoRecall(10)

Encontrados: 152
Total: 295
Acurácia 20 docs: 0.5152542372881356




In [227]:
avaliacaoRecall(20)

Duração: 89.905044
Encontrados: 177
Total: 295
Acurácia 20 docs: 0.6




### Precision

In [156]:
avaliacaoPrecision(5)

Precision: 0.088814


In [157]:
avaliacaoPrecision(10)

Precision: 0.051525


In [158]:
avaliacaoPrecision(20)

Precision: 0.029153


### MAP

In [159]:
avaliacaoMAP(5)

MAP: 0.002746


In [160]:
avaliacaoMAP(10)

MAP: 0.001597


In [161]:
avaliacaoMAP(20)

MAP: 0.001008


### RR

In [162]:
avaliacaoRR(5)

RR: 0.318644


In [228]:
data_preprocess = None

### 9 lowercase + punctuation, acetuation, and stopword removal + stemming (Savoy)

In [229]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = Savoy()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
        
    return terms

In [230]:
data_preprocess = [preprocess(doc) for doc in data]

In [231]:
a = time.time()

In [232]:
bm25 = BM25L(data_preprocess)

In [233]:
d = time.time()

In [234]:
print("Duração indexação: %f" % (d-a))

Duração indexação: 2.043986


### Recall

In [167]:
avaliacaoRecall(5)

Encontrados: 135
Total: 295
Acurácia 20 docs: 0.4576271186440678




In [168]:
avaliacaoRecall(10)

Encontrados: 152
Total: 295
Acurácia 20 docs: 0.5152542372881356




In [235]:
avaliacaoRecall(20)

Duração: 86.370035
Encontrados: 177
Total: 295
Acurácia 20 docs: 0.6




### Precision

In [170]:
avaliacaoPrecision(5)

Precision: 0.091525


In [171]:
avaliacaoPrecision(10)

Precision: 0.051525


In [172]:
avaliacaoPrecision(20)

Precision: 0.029661


### MAP

In [173]:
avaliacaoMAP(5)

MAP: 0.002812


In [174]:
avaliacaoMAP(10)

MAP: 0.001558


In [175]:
avaliacaoMAP(20)

MAP: 0.000878


### RR

In [176]:
avaliacaoRR(5)

RR: 0.308475


In [236]:
data_preprocess = None

### 18 lowercase + punctuation, acetuation, and stopword removal + stemming (RSLP) + unigram and bigram 

In [237]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = RSLPStemmer()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
    
    ngram = []
    ngram_1 = list(ngrams(terms, 1))
    ngram_2 = list(ngrams(terms, 2))
    for w in ngram_1:
        ngram.append(w[0])
        
    for w in ngram_2:
        string = w[0] + " " + w[1]
        ngram.append(string)
    
    return ngram


In [238]:
data_preprocess = [preprocess(doc) for doc in data]

In [239]:
a = time.time()

In [240]:
bm25 = BM25L(data_preprocess)

In [241]:
d = time.time()

In [242]:
print("Duração indexação: %f" % (d-a))

Duração indexação: 6.444999


### Recall

In [181]:
avaliacaoRecall(5)

Encontrados: 142
Total: 295
Acurácia 20 docs: 0.48135593220338985




In [182]:
avaliacaoRecall(10)

Encontrados: 149
Total: 295
Acurácia 20 docs: 0.5050847457627119




In [243]:
avaliacaoRecall(20)

Duração: 112.824045
Encontrados: 173
Total: 295
Acurácia 20 docs: 0.5864406779661017




### Precision

In [184]:
avaliacaoPrecision(5)

Precision: 0.096271


In [185]:
avaliacaoPrecision(10)

Precision: 0.050508


In [186]:
avaliacaoPrecision(20)

Precision: 0.028983


### MAP

In [187]:
avaliacaoMAP(5)

MAP: 0.002878


In [188]:
avaliacaoMAP(10)

MAP: 0.001537


In [189]:
avaliacaoMAP(20)

MAP: 0.000932


### RR

In [190]:
avaliacaoRR(5)

RR: 0.325424


In [244]:
data_preprocess = None

### 21 lowercase + punctuation, acetuation, and stopword removal + stemming (Savoy) + unigram and bigram

In [245]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))
    stemmer = Savoy()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
    
    ngram = []
    ngram_1 = list(ngrams(terms, 1))
    ngram_2 = list(ngrams(terms, 2))
    for w in ngram_1:
        ngram.append(w[0])
        
    for w in ngram_2:
        string = w[0] + " " + w[1]
        ngram.append(string)
    
    return ngram

In [246]:
data_preprocess = [preprocess(doc) for doc in data]

In [247]:
a = time.time()

In [248]:
bm25 = BM25L(data_preprocess)

In [249]:
d = time.time()

In [250]:
print("Duração indexação: %f" % (d-a))

Duração indexação: 7.199000


### Recall

In [195]:
avaliacaoRecall(5)

Encontrados: 143
Total: 295
Acurácia 20 docs: 0.4847457627118644




In [196]:
avaliacaoRecall(10)

Encontrados: 151
Total: 295
Acurácia 20 docs: 0.511864406779661




In [251]:
avaliacaoRecall(20)

Duração: 107.046000
Encontrados: 174
Total: 295
Acurácia 20 docs: 0.5898305084745763




### Precision

In [198]:
avaliacaoPrecision(5)

Precision: 0.096949


In [199]:
avaliacaoPrecision(10)

Precision: 0.051186


In [200]:
avaliacaoPrecision(20)

Precision: 0.028475


### MAP

In [201]:
avaliacaoMAP(5)

MAP: 0.002900


In [202]:
avaliacaoMAP(10)

MAP: 0.001560


In [203]:
avaliacaoMAP(20)

MAP: 0.000851


### RR

In [204]:
avaliacaoRR(5)

RR: 0.318644


### 22 lowercase + punctuation, acetuation, and stopword removal + stemming (RSLP_S)

In [252]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = RSLP_S()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
        
    return terms

In [253]:
data_preprocess = [preprocess(doc) for doc in data]

In [254]:
a = time.time()

In [255]:
bm25 = BM25L(data_preprocess)

In [256]:
d = time.time()

In [257]:
print("Duração indexação: %f" % (d-a))

Duração indexação: 2.309290


### Recall

In [98]:
avaliacaoRecall(5)

Encontrados: 132
Total: 295
Acurácia 20 docs: 0.44745762711864406




In [99]:
avaliacaoRecall(10)

Encontrados: 154
Total: 295
Acurácia 20 docs: 0.5220338983050847




In [258]:
avaliacaoRecall(20)

Duração: 88.837002
Encontrados: 175
Total: 295
Acurácia 20 docs: 0.5932203389830508




### Precision

In [101]:
avaliacaoPrecision(5)

Precision: 0.089492


In [102]:
avaliacaoPrecision(10)

Precision: 0.052203


In [103]:
avaliacaoPrecision(20)

Precision: 0.029322


### MAP

In [104]:
avaliacaoMAP(5)

MAP: 0.002670


In [105]:
avaliacaoMAP(10)

MAP: 0.001580


In [106]:
avaliacaoMAP(20)

MAP: 0.000873


### RR

In [107]:
avaliacaoRR(5)

RR: 0.294915


In [259]:
data_preprocess = None

### 23 lowercase + punctuation, acetuation, and stopword removal + stemming (RSLP) + unigram and bigram 

In [260]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = RSLP_S()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
    
    ngram = []
    ngram_1 = list(ngrams(terms, 1))
    ngram_2 = list(ngrams(terms, 2))
    for w in ngram_1:
        ngram.append(w[0])
        
    for w in ngram_2:
        string = w[0] + " " + w[1]
        ngram.append(string)
    
    return ngram


In [261]:
data_preprocess = [preprocess(doc) for doc in data]

In [262]:
a = time.time()

In [263]:
bm25 = BM25L(data_preprocess)

In [264]:
d = time.time()

In [265]:
print("Duração indexação: %f" % (d-a))

Duração indexação: 6.667005


### Recall

In [112]:
avaliacaoRecall(5)

Encontrados: 137
Total: 295
Acurácia 20 docs: 0.46440677966101696




In [113]:
avaliacaoRecall(10)

Encontrados: 152
Total: 295
Acurácia 20 docs: 0.5152542372881356




In [266]:
avaliacaoRecall(20)

Duração: 109.999998
Encontrados: 171
Total: 295
Acurácia 20 docs: 0.5796610169491525




### Precision

In [115]:
avaliacaoPrecision(5)

Precision: 0.092881


In [116]:
avaliacaoPrecision(10)

Precision: 0.051525


In [117]:
avaliacaoPrecision(20)

Precision: 0.028136


### MAP

In [118]:
avaliacaoMAP(5)

MAP: 0.002743


In [119]:
avaliacaoMAP(10)

MAP: 0.001553


In [120]:
avaliacaoMAP(20)

MAP: 0.000846


### RR

In [121]:
avaliacaoRR(5)

RR: 0.308475


In [122]:
data_preprocess = None